## BOAR with SIMsalabim
Version 0.1
(c) Larry Lueer, Vincent M. Le Corre, i-MEET 2021-2023

This notebook is made to use BOAR in combination with drift-diffusion modeling to fit of 'fake' JV curves.\
To perform the drift-diffusion simulation in the background we use the open-source program [SIMsalabim](https://github.com/kostergroup/SIMsalabim), for more information about SIMsalabim please check the [GitHub repository](https://github.com/kostergroup/SIMsalabim) \

Here we are fitting some 'fake' data that are generated by the drift-diffusion model.\

In [ ]:
# Activate matplotlib widgets
%matplotlib inline
# comment the next line if you are on the jupyterhub server
%matplotlib widget 
# %matplotlib notebook

# Import libraries
import sys
from numpy.random import default_rng
from scipy import interpolate
from sklearn.preprocessing import minmax_scale
import warnings
warnings.filterwarnings('ignore') # comment this out to see warnings

# Import boar
# import one folder up
sys.path.append('../') # comment out if the Notebook is in the Notebooks folder
from boar import *
# Import homemade package by VLC
# import boar.SIMsalabim_utils.plot_settings_screen # to set default plot settings

In [ ]:
# Define path to SIMsalabim
path2simu = os.path.join('/home/vlc/Desktop/' , 'SIMsalabim','SimSS') # path to the SIMsalabim directory
# Directory where the results are stored
curr_dir = os.getcwd()
res_dir = os.path.join(curr_dir,'temp') # path to the results directory, use this line if Notebook is in the Notebooks folder
# res_dir = os.path.join('../','temp') # path to the results directory, use this line if Notebook is in the Notebooks folder

In [ ]:
# define Fitparameters
True_params = {'kdirect':5e-18,'mun_0':2e-8,'mup_0':8e-8,'Nc':5e26,'Gehp':1.28e28,'W_L':0}#,'Rseries':3e-4,'Bulk_tr':1e20,'Gehp':1.28e28}
params = []

kdirect = Fitparam(name = 'kdirect', val = True_params['kdirect'] , relRange = 1.5, lims=[1e-18,1e-16],range_type='log',optim_type='log',display_name='k$_{2}$ [m$^{3}$ s$^{-1}$]') 
params.append(kdirect)
kdirect.startVal = 2e-18
mun_0 = Fitparam(name = 'mun_0', val = True_params['mun_0'] , relRange = 1.5, lims=[1e-8,1e-7],range_type='log',optim_type='log',display_name='$\mu_n$ [m$^{2}$ V$^{-1}$ s$^{-1}$]')
params.append(mun_0)
mun_0.startVal = 4e-8
mup_0 = Fitparam(name = 'mup_0', val = True_params['mup_0'] , relRange = 1.5, lims=[1e-8,1e-7],range_type='log',optim_type='log',display_name='$\mu_p$ [m$^{2}$ V$^{-1}$ s$^{-1}$]')
params.append(mup_0)
mup_0.startVal = 4e-8
# Nc = Fitparam(name = 'Nc', val = True_params['Nc'] , relRange = 1.5, lims=[1e26,1e27],range_type='log',optim_type='log',lim_type='absolute',axis_type='log',display_name='N$_{c}$ [m$^{-3}]$')
# params.append(Nc)
# Nc.startVal = 5e26
# W_L = Fitparam(name = 'W_L', val = True_params['W_L'] , relRange = 1.5, lims=[0,0.2],range_type='linear',optim_type='linear',lim_type='absolute',display_name='W$_{L}$ [eV]')
# params.append(W_L)

params_true = copy.deepcopy(params)


In [ ]:
# Define figures-of-merit (FOMs)
FOMs = []
FOMs.append(FOMparam(Theta_B, name = 'Theta_B', display_name='$\\theta_B$', optim_type = 'log'))
FOMs.append(FOMparam(delta_B, name = 'delta_B', display_name='$\delta_B$', optim_type = 'log'))


FOM_names = [fom.display_name for fom in FOMs] # names of the FOMs

## Prepare fake data for fitting
In the next block we create some fake data with some random noise and plot it.

In [ ]:
# # create some fake data
Nc = 1 # number of fake datasets
V = np.linspace(0,1,100) # voltage
Gfrac = np.asarray([0.01,0.5,1]) # Gfrac, i.e. light intensity

X_dimensions = ['Vext','Gfrac'] # dimensions of the X array
X = np.array([[x,y] for y in Gfrac for x in V ] ) # X array
Xplot = Gfrac # X array for plotting

# Generate the fake data to fit
degradation_run = True
True_paramsList = []

# define the degradation of kdirect
kvals = np.geomspace(5e-18,5e-17,Nc) # simulate degradation of kdirect

dda = Drift_diffusion_agent(path2simu=path2simu) # instantiate the agent

ys = []
True_vals = []
True_FOMs = []
for kval in kvals:
    kdirect.val = kval
    True_paramsList.append({'kdirect':kval})
    True_params['kdirect'] = kval
    # store the true values for plotting later
    True_vals.append(True_params.copy())
    FOM_dum = dda.get_FOM({'kdirect':kval,'mun_0':True_params['mun_0'],'mup_0':True_params['mup_0'],'Nc':True_params['Nc'],'Gehp':True_params['Gehp']}, FOMs) ##,'Nc':True_params['Nc']}
    idx = 0
    for fom,val in zip(FOMs,FOM_dum):
        if fom.optim_type=='log':
            FOM_dum[fom.name] = 10**FOM_dum[fom.name]
        else:
            FOM_dum[fom.name] = FOM_dum[fom.name]*fom.p0m
    True_FOMs.append(copy.deepcopy(FOM_dum))

    y = dda.DriftDiffusion_relative(X,params,X_dimensions=X_dimensions, max_jobs=3)
    rng = default_rng()#
    noise = rng.standard_normal(y.shape) * 1.9
    #noise = noise * X[:,1] # try this: higher T - higher noise  
    y+=noise # add some noise
    ys.append(y)
  
    plt.plot(X[:,0],y,'o')
plt.xlabel('Vext [V]')
plt.ylabel('Current Density [A/m$^2$]')
# save params list to be modified later to store true values
params_true = copy.deepcopy(params)


In [ ]:
# Fit the datasets one by one
pf,pf_fom = [],[]
mo = MultiObjectiveOptimizer(params=params,res_dir=res_dir)

n_jobs = 4 # was 4
n_jobs_init = 25 # was 4
n_yscale=20 # was 20
n_BO = 60 # was 60
n_initial_points = 60 # was 200
n_BO_warmstart = 60 # was 5

for ii,y in enumerate(ys):
    target = {'model':partial(dda.DriftDiffusion_relative,X_dimensions=X_dimensions,max_jobs=3),'target_name':'JV','data':{'X':X,'y':y,
                'X_dimensions':['Vext','Gfrac'],'X_units':['V','sun'],'y_dimension':'Current density','y_unit':r'$A/m²$'},'weight':1,'target_weight':1}
    mo.targets = [target]
    mo.params = params
    mo.warmstart = 'collect_init' if ii==0 else 'recall'
    # mo.warmstart = 'None'
    mo.SaveOldXY2file = os.path.join(res_dir,'old_XY.json') # path to the file where old points are saved
    mo.Path2OldXY = os.path.join(res_dir,'old_XY.json') # path to the file where old points are saved
   
    kwargs = {'check_improvement':'relax','max_loop_no_improvement':10,'xtol':1e-3,'ftol':1e-3}
    kwargs_posterior = {'Nres':10,'gaussfilt':3,'logscale':True,'vmin':1e-100,'zoom':0,'min_prob':1e-40,'clear_axis':False,'True_values':True_vals[ii],'show_points':True,'savefig':False,'figname':'param_posterior'+str(ii)}
    kwargs_plot_obj = {'zscale':'linear'}

    r = mo.optimize_sko_parallel(n_jobs=n_jobs,n_yscale=n_yscale, n_BO=n_BO, n_initial_points = n_initial_points,n_BO_warmstart=n_BO_warmstart,n_jobs_init=n_jobs_init,kwargs=kwargs,verbose=False,loss='linear',threshold=1000,base_estimator = 'GP',show_objective_func=False,show_posterior=True,kwargs_posterior = kwargs_posterior,kwargs_plot_obj=kwargs_plot_obj)
    pf.append(deepcopy(mo.params)) # collects optimized fitparameters
    rrr = r['r'] # the results dict of the last optimizer.tell()

    FOM_list = dda.get_FOMs(rrr.x_iters,params,FOMs)
    
    for idx,fom in enumerate(FOMs): # update the FOMs with the optimized values
        fom.update_FOMparam(FOM_list[:,idx])
    

    kwargs_plot_obj = {'zscale':'linear'}
    kwargs_posterior = {'Nres':5,'gaussfilt':3,'logscale':True,'vmin':1e-100,'zoom':0,'min_prob':1e-40,'clear_axis':True,'show_points':True,'True_values':True_FOMs[ii],'savefig':False,'figname':'FOM_posterior'+str(ii)}
    FOM_min,FOM_std = mo.single_point(FOM_list.tolist(),list(rrr.func_vals),FOMs,n_jobs=4,base_estimator='GP',n_initial_points = 50,show_objective_func=False,kwargs_plot_obj=kwargs_plot_obj,show_posterior=True,kwargs_posterior = kwargs_posterior)
    for i in range(len(FOMs)):
        if FOMs[i].optim_type=='log':
            FOMs[i].val = 10**FOM_min[i]  
        else:
            FOMs[i].val = FOM_min[i]
        FOMs[i].std = FOM_std[i]

    pf_fom.append(deepcopy(FOMs)) # collects optimized FOMs

    #get true parameters and make a params object
    for param in params_true:
        if param.name in True_paramsList[ii]:
            param.val = True_paramsList[ii][param.name]

    # plot the fit results
    fit_results = []
    kwargs_plot_res = {'x_scaling':1,'xaxis_label':'Voltage [V]','xscale_type':'linear','y_scaling':10,'yaxis_label':'Current density [mA cm$^2$]','yscale_type':'linear','norm_data':False,'delog':False,'figsize':(10,10),'savefig':False,'figname':'JV_fits_' + str(ii),'figdir':'temp'}

    for num,t in enumerate(mo.targets):
        kwargs_plot_res['figname'] = os.path.join(res_dir,t['target_name']+f'_fit_{num}')
        dda.plot_fit_res(t,mo.params,'Vext',xlim=[],ylim=[],kwargs=kwargs_plot_res)

        X = t['data']['X']
        y = t['data']['y']
        X_dimensions = t['data']['X_dimensions']
        yfit = t['model'](X,params,X_dimensions=X_dimensions) # get the best fits

        data = np.concatenate((X, y.reshape(len(y),1), yfit.reshape(len(yfit),1)), axis=1)
        fit_results.append(data)


In [ ]:
# plot the parameters evolution
param_plot = dda.plot_params(pf,kwargs={'savefig':True,'figname':'plot_param','figdir':'temp','nrows':2,'ncols':3,'figsize':(10,10)})
fom_plot = dda.plot_params(pf_fom,kwargs={'savefig':True,'figname':'plot_FOM','figdir':'temp','nrows':1,'ncols':2,'figsize':(10,10)})

In [ ]:
# Clean output files from simulation folders
from boar.SIMsalabim_utils.CleanFolder import *
# path2simu = '/home/vlc/Desktop/PVLC_notebook/Simulation_program/SIMsalabimv433/SimSS'
Do_Cleaning = True # Careful, this will delete all files in the folder
if Do_Cleaning:
    clean_up_output('tj',path2simu)
    clean_up_output('tVG',path2simu)
    clean_up_output('JV',path2simu)
    clean_up_output('Var',path2simu)
    clean_up_output('scPars',path2simu)
    clean_up_output('Str4Parallel',path2simu)
    # os.remove(mo.path2oldxy) # remove the old_xy.json file if it exists
    # delete warmstart folder if it exists
    if os.path.exists(os.path.join(os.getcwd(),'warmstart/')):
        shutil.rmtree(os.path.join(os.getcwd(),'warmstart/'))
    # delete temp folder if it exists
    # if os.path.exists(os.path.join(os.getcwd(),'temp/')):
    #     shutil.rmtree(os.path.join(os.getcwd(),'temp/'))

